In [25]:
import matplotlib.pyplot as plt
import numpy as np
import schemdraw
import schemdraw.elements as elm
from sympy import (
    Derivative,
    Eq,
    Function,
    atan,
    cos,
    exp,
    lambdify,
    sin,
    solve,
    sqrt,
    symbols,
)
from sympy import I as j

plt.rcParams.update({"text.usetex": True})

# Life With and Without Phasors

One of the main reasons for many people being confused about phasors is that phasors are abstract, but they don't understand why the abstraction exists. In other words, they may know what phasors *represent*, but they don't know what they *replace*---they don't know of any alternative approach for analyzing AC circuits that can be used to put phasors in perspective. This is understandable as many electrical engineering textbooks poorly explain why we use phasors.

With the aid of an example AC circuit, this section goes over what's hidden behind the phasor abstraction. By first trying to solve the circuit without phasors, we will better understand how AC circuit analysis works at its core. And by then solving the circuit with phasors, we will see that it is not only a more concise notation, but also a simpler mathematical representation and a faster method for analyzing AC circuits.

The following AC circuit is our example circuit. Note that it has multiple AC sources and an bridge-type topology that does not allow us to consolidate impedances using formulae for impedances in series or in parallel.

In [ ]:
fig, ax = plt.subplots(layout="tight")
ax.set_aspect("equal")
ax.set_axis_off()
with schemdraw.Drawing(canvas=ax):
    v_s1 = elm.SourceSin().up(8).label(["−", r"$v_{s1}$", "+"])
    elm.Line().right().dot()
    L = elm.Inductor().down(4).label(["+", r"$L$  $v_\mathrm{L}$", "−"])
    elm.CurrentLabel(length=1, ofst=0.3).at(L).label(r"$i_\mathrm{L}$")
    R2 = elm.Resistor().down(4).label(["+", r"$R_2$  $v_\mathrm{R2}$", "−"]).dot()
    elm.CurrentLabel(length=1, ofst=0.3).at(R2).label(r"$i_\mathrm{R2}$")
    elm.Ground()
    elm.Line().left()
    elm.Line().right(4).at(L.start)
    R1 = elm.Resistor().down(4).label(["+", r"$v_\mathrm{R1}$  $R_1$", "−"], "bottom").dot()
    elm.CurrentLabel(length=1, ofst=0.3, top=False).at(R1).label(r"$i_\mathrm{R1}$", "top")
    C = elm.Capacitor().down(4).label(["+", r"$v_\mathrm{C}$  $C$", "−"], "bottom")
    elm.CurrentLabel(length=1, ofst=0.3, top=False).at(C).label(r"$i_\mathrm{C}$", "top")
    elm.Line().left(4)
    elm.Dot().at(L.end).label("$v_a$", "left")
    elm.SourceSin().right(4).label(["+", r"$v_{s2}$", "−"]).dot().label("$v_b$", "right")
fig.savefig("img/fig_3_1.png", dpi=200)

```{image} img/fig_3_1.png
:align: center
:width: 65%
```

Say that we want to solve for voltages $v_a(t)$ and $v_b(t)$. Doing so would allow us to solve for the voltage across---and therefore the current through---any one component. Just as for a purely resistive DC network, Kirchhoff's Current Law (KCL) applies and we can do [nodal analysis](https://en.wikipedia.org/wiki/Nodal_analysis) to determine the voltages at nodes $a$ and $b$. In particular we must use the [supernode technique](https://en.wikipedia.org/wiki/Nodal_analysis#Supernodes) with a single node $ab$ because the current through voltage source $v_{s2}$ cannot be put in terms of $v_a(t)$ or $v_b(t)$. This loses us an equation, but we gain one in its place with the following relationship:

$$
v_{s2}(t) = v_a(t) - v_b(t)
$$ (v_s2_eq)

Let's try solving the circuit using node voltage analysis, with and without phasors.

In [27]:
V_s1, V_s2, w = symbols("V_s1, V_s2, omega", real=True)
R_1, R_2, L, C = symbols("R_1, R_2, L, C", real=True)


def subs(x):
    return x.subs([(V_s1, 1), (V_s2, 2), (w, 3), (R_1, 4), (R_2, 5), (L, 6), (C, 7)])

## Solving the circuit without phasors

Using KCL, the sum of currents flowing out of supernode $ab$ must sum to zero. We can therefore write:

$$ -i_\mathrm{L}(t) + i_\mathrm{R2}(t) - i_\mathrm{R1}(t) + i_\mathrm{C}(t) = 0 $$

Substituting in the components' characteristic equations gives us:

$$ -\frac{1}{L} \int_0^t v_\mathrm{L}(\tau) \, \mathrm{d}\tau + \frac{v_\mathrm{R2}(t)}{R_2} - \frac{v_\mathrm{R1}(t)}{R_1} + C \frac{\mathrm{d} v_\mathrm{C}(t)}{\mathrm{d}t} = 0 $$

Putting everything in terms of the node voltages yields:

$$
-\frac{1}{L} \int_0^t (v_{s1}(\tau) - v_a(\tau)) \, \mathrm{d}\tau + \frac{v_a(t)}{R_2} - \frac{v_{s1}(t) - v_b(t)}{R_1} + C \frac{\mathrm{d} v_b(t)}{\mathrm{d}t} = 0
$$ (kcl_integro_differential_eq)

Paired with Equation {eq}`v_s2_eq`, this represents a system of two integro-differential equations. By differentiating, we put this in the more familiar form of a system of differential equations (in which the highest-order derivative is two):

$$
\begin{aligned}
& \! -\frac{1}{L} (v_{s1}(t) - v_a(t)) + \frac{1}{R_2} \frac{\mathrm{d} v_a(t)}{\mathrm{d}t} - \frac{1}{R_1} \frac{\mathrm{d}}{\mathrm{d}t} (v_{s1}(t) - v_b(t)) + C \frac{\mathrm{d}^2 v_b(t)}{{\mathrm{d}t}^2} = 0 \\
& v_{s2}(t) = v_a(t) - v_b(t)
\end{aligned}
$$

We put this into a canonical form of a system of first-order differential equations by introducing an auxiliary variable $v_b'(t)$ and by using Equation {eq}`v_s2_eq` to replace the need for variable $v_a(t)$ with $v_b'(t)$:

$$
\begin{aligned}
& \frac{\mathrm{d} v_b(t)}{\mathrm{d}t} = v_b'(t) \\
& \frac{\mathrm{d} v_b'(t)}{\mathrm{d}t} = \frac{1}{C} \left[ \frac{v_{s1}(t) - (v_{s2}(t) + v_b(t))}{L} - \frac{v_{s2}'(t) + v_b'(t)}{R_2} + \frac{v_{s1}'(t) - v_b'(t)}{R_1} \right]
\end{aligned}
$$

We specify that $v_{si}(t) = V_{si} \sin \omega t$ (and $v_{si}'(t) = \omega V_{si} \cos \omega t$):

$$
\begin{aligned}
& \frac{\mathrm{d} v_b(t)}{\mathrm{d}t} = v_b'(t) \\
& \frac{\mathrm{d} v_b'(t)}{\mathrm{d}t} = \frac{1}{C} \left[ \frac{(V_{s1} - V_{s2}) \sin \omega t - v_b(t)}{L} - \frac{\omega V_{s2} \cos \omega t + v_b'(t)}{R_2} + \frac{\omega V_{s1} \cos \omega t - v_b'(t)}{R_1} \right]
\end{aligned}
$$

In [28]:
# Trying to solve the system of differential equations using SymPy's `dsolve`.

t = symbols("t", real=True)
v_s1 = V_s1 * sin(w * t)
v_s2 = V_s2 * sin(w * t)

v_s1p = Derivative(v_s1, t).simplify()
v_s2p = Derivative(v_s2, t).simplify()

v_b = Function("v_b")(t)
v_bp = Function("v_bp")(t)

eq1 = Eq(Derivative(v_b, t), v_bp)
eq2 = Eq(
    Derivative(v_bp, t),
    ((v_s1 - (v_s2 + v_b)) / L - (v_s2p + v_bp) / R_2 + (v_s1p - v_bp) / R_1) / C,
)

# result = dsolve([eq1, eq2], [v_b, v_bp])  # Hangs.

# https://docs.sympy.org/latest/guides/solving/solve-ode.html

Our system is now in terms of two unknowns---$v_b(t)$ and $v_b'(t)$---two functions to solve for. We start to solve our system of differential equations by assuming a solution of the form:

$$
\begin{aligned}
v_b(t) & = V_b \cos(\omega t + \theta_b) \\
v_b'(t) & = -\omega V_b \sin(\omega t + \theta_b)
\end{aligned}
$$

By substituting into our system of differential equations, we convert it to a regular system of equations---or, in this case, just one equation:

$$
-\omega^2 V_b \cos(\omega t + \theta_b) = \frac{1}{C} \left[
\begin{aligned}
& \frac{(V_{s1} - V_{s2}) \sin \omega t - V_b \cos(\omega t + \theta_b)}{L} \\
& + \omega \left[ \left( \frac{V_{s1}}{R_1} - \frac{V_{s2}}{R_2} \right) \cos \omega t + \left ( \frac{1}{R_1} + \frac{1}{R_2} \right) V_b \sin(\omega t + \theta_b) \right]
\end{aligned}
\right]
$$

This equation is in terms of two unknowns---$V_b$ and $\theta_b$. While having only one equation may appear to be insufficient to solve for two unknowns, the equation is parameterized by $t$ and must be satisfied for all values of $t$. So, in this sense, we actually have an infinite number of equations at our disposal. We can pick two 'easy' values of $t$ that result in two relatively simple equations:

$t = 0$:

$$
-\omega^2 C V_b \cos \theta_b = \frac{- V_b \cos \theta_b}{L} + \omega \left[ \left( \frac{V_{s1}}{R_1} - \frac{V_{s2}}{R_2} \right) + \left ( \frac{1}{R_1} + \frac{1}{R_2} \right) V_b \sin \theta_b \right]
$$

$t = -\theta_b / \omega$:

$$
-\omega^2 C V_b = \frac{- (V_{s1} - V_{s2}) \sin \theta_b - V_b}{L} + \omega \left( \frac{V_{s1}}{R_1} - \frac{V_{s2}}{R_2} \right) \cos \theta_b
$$

<!-- $$ V_b = \frac{\omega \left( \frac{V_{s1}}{R_1} - \frac{V_{s2}}{R_2} \right)}{\left( \frac{1}{L} - \omega^{2} C \right) \cos \theta_b - \omega \left( \frac{1}{R_1} + \frac{1}{R_2} \right) \sin \theta_b} $$ -->
<!-- $$ V_b = \frac{-\frac{V_{s1} - V_{s2}}{L} \sin \theta_b + \omega \left( \frac{V_{s1}}{R_1} - \frac{V_{s2}}{R_2} \right) \cos \theta_b}{\frac{1}{L} - \omega^{2} C} $$ -->

```{note}
There is technically an infinite number of solutions for $\theta_b$, because any multiple of $2 \pi$ can be added to (or subtracted from) $\theta_b$, resulting in an identical sinusoid with a different equation.
```

In [ ]:
theta_b_xs = np.linspace(-3, +3, 10000)

theta_b = symbols("theta_b")

V_b_eq1 = (w * (V_s1 / R_1 - V_s2 / R_2)) / (
    (1 / L - w**2 * C) * cos(theta_b) - w * (1 / R_1 + 1 / R_2) * sin(theta_b)
)
V_b_eq2 = (
    -(V_s1 - V_s2) / L * sin(theta_b) + w * (V_s1 / R_1 - V_s2 / R_2) * cos(theta_b)
) / (1 / L - w**2 * C)

plt.plot(theta_b_xs, lambdify(theta_b, subs(V_b_eq1))(theta_b_xs), ".", markersize=1)
plt.plot(theta_b_xs, lambdify(theta_b, subs(V_b_eq2))(theta_b_xs), ".", markersize=1)
plt.xlim(-3, 3)
plt.ylim(-0.1, 0.1)

sol = (theta_b_sol, V_b_sol) = (-0.333224, 0.007635)
plt.plot([theta_b_sol], [V_b_sol], "ok")
plt.annotate(f"${sol}$", sol, textcoords="offset points", xytext=(20, 10), ha="center")

plt.savefig("img/fig_3_2.png", dpi=200)

<!-- https://www.desmos.com/calculator/1qjx4bylgv -->
```{image} img/fig_3_2.png
:align: center
:width: 65%
```

This is still a difficult nonlinear system to solve. Furthermore, as we have seen, the process to derive the system of equations is long and notation-heavy despite being consistent between different AC circuits.

~~Now that we are familiar with the non-phasor approach to solving the AC circuit, let's solve the circuit with phasors to see how it compares.~~

## Solving the circuit with the Laplace transform

The Laplace transform $\mathcal{L}$ converts a function $f(t)$ (in the *time domain*) into a function $F(s)$ (in the *complex frequency domain*) through the following integration:

$$
F(s) = \mathcal{L}\{ f \}(s) = \int_0^\infty f(t) \, \mathrm{e}^{-s t} \, \mathrm{d} t
$$ (laplace_def)

The Laplace transform is closely related to the Fourier transform. Both transforms convert to the frequency domain. However, the Laplace transform converts to a frequency domain whose variable $s$ is a *complex number*, which can simplify calculations. This domain is known as the *$s$-domain* or *Laplace domain*.

The Laplace transform has a number of favorable properties. One such property is that differentiating with respect to $t$ in the time domain becomes multiplication by $s$ in the Laplace domain. Similarly, integrating with respect to $t$ in the time domain becomes division by $s$ in the Laplace domain. Because of this, the Laplace transform is often used to convert a system of differential equations into a regular system of equations which can be solved more easily.

Furthermore, many useful functions in the time domain become simple, rational functions in the Laplace domain, for example:

$$
\begin{aligned}
& \text{Sine wave:} && \mathcal{L}\{ \sin(\omega t) \, u(t) \} = \frac{\omega}{s^2 + \omega^2} \\
& \text{Phase-shifted cosine wave:} && \mathcal{L}\{ \cos(\omega t + \theta) \, u(t) \} = \frac{s \cos \theta - \omega \sin \theta}{s^2 + \omega^2} \\
& \text{Exponential decay:} && \mathcal{L}\{ \mathrm{e}^{-\alpha t} \, u(t) \} = \frac{1}{s + \alpha} \\
& \text{Exponentially decaying cosine wave:} && \mathcal{L}\{ \mathrm{e}^{-\alpha t} \cos(\omega t) \, u(t) \} = \frac{s + \alpha}{(s + \alpha)^2 + \omega^2}
\end{aligned}
$$

Listings of the Laplace transforms of common functions, such as those above, are called Laplace tables.

$u(t)$ is the Heaviside step function, which is $1$ for $t \geq 0$ and $0$ otherwise. The time-domain functions are multiplied by $u(t)$---that is, only nonzero for $t \geq 0$, for the same reason that the lower bound of integration in Equation {eq}`laplace_def` is $0$. The Laplace transform only looks at $t \geq 0$, and is thus useful for solving differential equations involving functions that are only nonzero for $t \geq 0$. This aspect allows us to answer questions about a system---particularly an electric circuit---that is modeled by differential equations, such as: "what happens when the system is turned on?", "what happens when the value of this component suddenly changes?", and "what happens when the circuit's topology changes in this way?" To consider functions that are nonzero for $t < 0$, and thus model no sudden change at $t = 0$, the *two-sided* or *bilateral* Laplace transform can be used.

We want to solve the system of two integro-differential equations {eq}`v_s2_eq` and {eq}`kcl_integro_differential_eq` that we derived in the previous section. That system was in terms of $v_a(t)$ and $v_b(t)$, in the time domain. First, we will apply the Laplace transform to convert it to a regular system of equations in terms of $V_a(s)$ and $V_b(s)$, in the Laplace domain. Then, we will be able to solve for $V_a(s)$ and $V_b(s)$ very easily. Finally, we will convert our solution back to the time domain using the inverse Laplace transform.

Applying the Laplace transform to equations {eq}`v_s2_eq` and {eq}`kcl_integro_differential_eq` gives us:

$$
\begin{aligned}
& \! -\frac{V_{s1}(s) - V_a(s)}{L s} + \frac{V_a(s)}{R_2} - \frac{V_{s1}(s) - V_b(s)}{R_1} + C s V_b(s) = 0 \\
& V_{s2}(s) = V_a(s) - V_b(s)
\end{aligned}
$$

Here, $V_{s1}(s)$ and $V_{s2}(s)$ are the Laplace transforms of $v_{s1}(t)$ and $v_{s2}(t)$---not to be confused with the amplitudes $V_{s1}$ and $V_{s2}$ of $v_{s1}(t)$ and $v_{s2}(t)$.

Solving for $V_a(s)$ in the second equation and substituting it into the first equation yields:

$$
-\frac{V_{s1}(s) - (V_{s2}(s) + V_b(s))}{L s} + \frac{V_{s2}(s) + V_b(s)}{R_2} - \frac{V_{s1}(s) - V_b(s)}{R_1} + C s V_b(s) = 0
$$

Solving for $V_b(s)$ gives us:

<!-- $$ -\frac{1}{L s} V_b(s) + \frac{1}{R_2} V_b(s) - \frac{1}{R_1} V_b(s) + C s V_b(s) = \frac{V_{s1}(s) - V_{s2}(s)}{L s} - \frac{V_{s2}(s)}{R_2} + \frac{V_{s1}(s)}{R_1} $$ -->
<!-- $$ V_b = \frac{\dfrac{V_{s1}(s) - V_{s2}(s)}{L s} - \dfrac{V_{s2}(s)}{R_2} + \dfrac{V_{s1}(s)}{R_1}}{-\dfrac{1}{L s} + \dfrac{1}{R_2} - \dfrac{1}{R_1} + C s} $$ -->
<!-- $$ V_b = \frac{R_1 R_2 (V_{s1}(s) - V_{s2}(s)) - R_1 L s V_{s2}(s) + R_2 L s V_{s1}(s)}{R_1 R_2 + R_1 L s + R_2 L s + R_1 R_2 L C s^2} $$ -->

$$ V_b = \frac{R_2 (R_1 + L s) V_{s1}(s) - R_1 (R_2 + L s) V_{s2}(s)}{R_1 R_2 + (R_1 + R_2) L s + R_1 R_2 L C s^2} $$

Substituting in $V_{si}(s) = \mathcal{L}\{ V_{si} \sin(\omega t) \, u(t) \} = V_{si} \, \omega / (s^2 + \omega^2)$ yields the following. Note that, because the voltage sources are multiplied by $u(t)$, we are essentially answering the question of how the circuit responds when turned on at $t = 0$.

<!-- $$ V_b = \frac{R_2 (R_1 + L s) V_{s1} \dfrac{\omega}{s^2 + \omega^2} - R_1 (R_2 + L s) V_{s2} \dfrac{\omega}{s^2 + \omega^2}}{R_1 R_2 + (R_1 + R_2) L s + R_1 R_2 L C s^2} $$ -->

$$ V_b = \frac{R_2 (R_1 + L s) V_{s1} \omega - R_1 (R_2 + L s) V_{s2} \omega}{(s^2 + \omega^2) (R_1 R_2 + (R_1 + R_2) L s + R_1 R_2 L C s^2)} $$

In [ ]:
# Checking the result.

s = symbols("s")

V_s1_lap = V_s1 * w / (s**2 + w**2)
V_s2_lap = V_s2 * w / (s**2 + w**2)

V_a_lap = Function("V_a_lap")(s)
V_b_lap = Function("V_b_lap")(s)

eq1_lap = Eq(
    -(V_s1_lap - V_a_lap) / (L * s) + (V_s2_lap + V_b_lap) / R_2 - (V_s1_lap - V_b_lap) / R_1 + C * s * V_b_lap, 0
)
eq2_lap = Eq(V_s2_lap, V_a_lap - V_b_lap)

V_a_lap_sol, V_b_lap_sol = solve([eq1_lap, eq2_lap], [V_a_lap, V_b_lap]).values()

V_b_lap_sol.factor()

At this point, the algebra will be significantly easier if we substitute in known values for our parameters. Let's introduce some arbitrary values, after which we will omit units for brevity. However, $V_b(s)$ is still in units of volts and $s$ is still, like $\omega$, in units of radians per second.

In [ ]:
subs(V_b_lap_sol).factor()

$$
\begin{aligned}
V_b(s)
& = \frac{(5 \ \Omega) (4 \ \Omega + (6 \ \mathrm{H}) s) (1 \ \mathrm{V}) (3 \ \mathrm{rad} / \mathrm{s}) - (4 \ \Omega) (5 \ \Omega - (6 \ \mathrm{H}) s) (2 \ \mathrm{V}) (3 \ \mathrm{rad} / \mathrm{s})}{(3 \ \mathrm{rad} / \mathrm{s}) (- (4 \ \Omega) (5 \ \Omega) + (4 \ \Omega - 5 \ \Omega) (6 \ \mathrm{H}) s + (4 \ \Omega) (5 \ \Omega) (6 \ \mathrm{H}) (7 \ \mathrm{F}) s^2)} \\
& = \frac{-54 s - 60}{(s^2 + 9) (840 s^2 + 54 s + 20)}
\end{aligned}
$$

We need to convert this solution for $V_b(s)$ back to the time domain. Converting a function from the Laplace domain to the time domain can be done by looking it up in a Laplace table. However, $V_b(s)$ is too complicated in its current form to find a matching function in a Laplace table---in particular, because the order of the polynomial in the denominator is four. This is higher than the highest-order denominator of two that can be found in typical Laplace tables. We can handle this by rewriting our solution as a linear combination of simpler functions that we will be able to find in the Laplace table. To determine such simpler functions, we factor the denominator then apply *partial fraction decomposition*. The denominator needs to be split into factors whose order is at most two. This is already the case. Next, we assume the following modified format of our $V_b$ solution and find the values $A_1$ through $A_4$ that make it true:

<!-- https://ocw.mit.edu/courses/18-03sc-differential-equations-fall-2011/46cc7780ac785d64f294b67fc2357330_MIT18_03SCF11_s28_2text.pdf -->

In [ ]:
A_1, A_2, A_3, A_4 = symbols("A_1, A_2, A_3, A_4")

num1 = A_1 * s + A_2
den1 = s**2 + 9
num2 = A_3 * s + A_4
den2 = 840 * s**2 + 54 * s + 20

Eq(
    subs(V_b_lap_sol).as_numer_denom()[0],
    num1 * den2 + num2 * den1,
).expand().as_poly(s)

$$
\begin{aligned}
& V_b(s) = \frac{A_1 s + A_2}{s^2 + 9} + \frac{A_3 s + A_4}{840 s^2 + 54 s + 20} \\
& \implies \frac{-54 s - 60}{(s^2 + 9) (840 s^2 + 54 s + 20)} = \frac{A_1 s + A_2}{s^2 + 9} + \frac{A_3 s + A_4}{840 s^2 + 54 s + 20} \\
& \implies -54 s - 60 = (A_1 s + A_2) (840 s^2 + 54 s + 20) + (A_3 s + A_4) (s^2 + 9) \\
&
\begin{aligned}
\implies \,
& (-840 A_1 - A_3) s^3 \\
& + (-54 A_1 - 840 A_2 - A_4) s^2 \\
& + (-20 A_1 - 54 A_2 - 9 A_3 - 54) s \\
& + (-20 A_2 - 9 A_4 - 60) \\
& = 0
\end{aligned} \\
& \implies \left\{
\begin{aligned}
-840 A_1 - A_3 & = 0 \\
-54 A_1 - 840 A_2 - A_4 & = 0 \\
-20 A_1 - 54 A_2 - 9 A_3 - 54 & = 0 \\
-20 A_2 - 9 A_4 - 60 & = 0
\end{aligned}
\right. \\
& \implies \left\{
\begin{aligned}
& A_1 = & 102600 / 14219461 & \\
& A_2 = & 106539 / 14219461 & \\
& A_3 = & -86184000 / 14219461 & \\
& A_4 = & -95033160 / 14219461 &
\end{aligned}
\right.
\end{aligned}
$$

In [ ]:
A_1_sol, A_2_sol, A_3_sol, A_4_sol = solve(
    [
        Eq(-840 * A_1 - A_3, 0),
        Eq(-54 * A_1 - 840 * A_2 - A_4, 0),
        Eq(-20 * A_1 - 54 * A_2 - 9 * A_3 - 54, 0),
        Eq(-20 * A_2 - 9 * A_4 - 60, 0),
    ],
    [A_1, A_2, A_3, A_4],
).values()
A_1_sol, A_2_sol, A_3_sol, A_4_sol

In [ ]:
V_b_lap_sol_subs_apart = (
    num1.subs([(A_1, A_1_sol), (A_2, A_2_sol)]) / den1
    + num2.subs([(A_3, A_3_sol), (A_4, A_4_sol)]) / den2
)
assert V_b_lap_sol_subs_apart.simplify().factor() == subs(V_b_lap_sol).factor()
assert Eq(V_b_lap_sol_subs_apart, subs(V_b_lap_sol).apart()).simplify()
V_b_lap_sol_subs_apart

Substituting $A_1$ through $A_4$ gives us:

$$ V_b(s) = \underbrace{\frac{\frac{102600}{14219461} s + \frac{106539}{14219461}}{s^2 + 9}}_\text{First term} + \underbrace{\frac{-\frac{86184000}{14219461} s - \frac{95033160}{14219461}}{840 s^2 + 54 s + 20}}_\text{Second term} $$

**Inverse Laplace transform of the first term.** The first term matches the following Laplace table function:

$$ \mathcal{L}\{ \cos(\omega t + \theta) \, u(t) \} = \frac{s \cos \theta + \omega \sin \theta}{s + \omega^2} $$

Therefore:

$$ \text{First term} = \frac{\frac{102600}{14219461} s + \frac{106539}{14219461}}{s^2 + 9} = B_1 \frac{s \cos \theta_1 - \omega_1 \sin \theta_1}{s + \omega_1^2} $$

The $\omega_1$ here is (not coincidentally) the same as our natural frequency $\omega = 3 \ \mathrm{rad} / \mathrm{s}$. Now, we need to solve for $\theta_1$ and $B_1$, which we can do by setting up the following system of equations:

$$
\begin{aligned}
B_1 \cos \theta_1 & = 102600 / 14219461 \\
-3 B_1 \sin \theta_1 & = 106539 / 14219461
\end{aligned}
$$

We can solve for $\theta_1$ by dividing the second equation by the first:

$$ \frac{106539 / 14219461}{102600 / 14219461} = -3 \tan \theta_1 \implies \theta_1 = \arctan \left( -\frac{35513}{102600} \right) = -19.092^\circ $$

And we can solve for $B_1$ by dividing the second equation by $-3$ and adding its square to the square of the first:

$$
\begin{aligned}
& \left( \frac{102600}{14219461} \right)^2 + \left( \frac{106539}{-3 \cdot 14219461} \right)^2 = B_1^2 \cos^2 \theta_1 + B_1^2 \sin^2 \theta_1 = B_1^2 \\
& \implies B_1 = \sqrt{\left( \frac{102600}{14219461} \right)^2 + \left( \frac{35513}{14219461} \right)^2} = 0.007635
\end{aligned}
$$

Therefore, the inverse Laplace transform of the first term is:

$$ \mathcal{L}^{-1}\left\{ \frac{\frac{102600}{14219461} s + \frac{106539}{14219461}}{s^2 + 9}\right \} = 0.007635 \cos(3 t - 19.092^\circ) \, u(t) $$

In [ ]:
theta_1 = atan(A_2_sol / A_1_sol / -3)
print(f"theta_1 = {np.rad2deg(float(theta_1)):.3f} deg")
B_1 = sqrt(A_1_sol**2 + (A_2_sol / -3)**2)
print(f"B_1 = {float(B_1):.6f} V")

```{admonition} Derivation: Laplace transform of an exponentially decaying, phase-shifted cosine wave
$$
\begin{aligned}
\mathcal{L}\{ \mathrm{e}^{-\alpha t} \cos(\omega t + \theta) \, u(t) \}
& = \mathcal{L}\{ \mathrm{e}^{-\alpha t} \, [\cos \omega t \cos \theta - \sin \omega t \sin \theta] \, u(t) \} \\
& = \cos \theta \, \mathcal{L}\{ \mathrm{e}^{-\alpha t} \cos \omega t \, u(t) \} - \sin \theta \, \mathcal{L}\{ \mathrm{e}^{-\alpha t} \sin \omega t \, u(t) \} \\
& = \cos \theta \frac{s + \alpha}{(s + \alpha)^2 + \omega^2} - \sin \theta \frac{\omega}{(s + \alpha)^2 + \omega^2} \\
& = \frac{(s + \alpha) \cos \theta - \omega \sin \theta}{(s + \alpha)^2 + \omega^2}
\end{aligned}
$$ (exp_phase_cos_lap)
```

**Inverse Laplace transform of the second term.** The second term matches the following function from {eq}`exp_phase_cos_lap`:

$$ \mathcal{L}\{ \mathrm{e}^{-\alpha t} \cos(\omega t + \theta) \, u(t) \} = \frac{(s + \alpha) \cos \theta - \omega \sin \theta}{(s + \alpha)^2 + \omega^2} $$

Therefore:

$$
\begin{aligned}
& \text{Second term} = \frac{-\frac{86184000}{14219461} s - B_2 \frac{95033160}{14219461}}{840 s^2 + 54 s + 20} = \frac{(s + \alpha_2) \cos \theta_2 - \omega_2 \sin \theta_2}{(s + \alpha_2)^2 + \omega_2^2} \\
& \implies \frac{-\frac{102600}{14219461} s - \frac{791943}{99536227}}{s^2 + \frac{9}{140} s + \frac{1}{42}} = B_2 \frac{s \cos \theta_2 + (\alpha_2 \cos \theta_2 - \omega_2 \sin \theta_2)}{s^2 + 2 \alpha_2 s + (\alpha_2^2 + \omega_2^2)}
\end{aligned}
$$

We need to solve for $\alpha_2$, $\omega_2$, $\theta_2$, and $B_2$:

$$
\begin{aligned}
& 9 / 140 = 2 \alpha_2 \implies \alpha_2 = 9 / 280 = 0.03214 \\
& \alpha_2^2 + \omega_2^2 = \frac{1}{42} \implies \omega_2 = \sqrt{\frac{1}{42} - \left( \frac{9}{280} \right)^2} = 0.15092 \ \mathrm{rad} / \mathrm{s}
\end{aligned}
$$

We can solve for $\theta_2$ and $B_2$ by setting up and solving the following system of equations:

$$
\left.
\begin{aligned}
& -102600 / 14219461 = B_2 \cos \theta_2 \\
& -791943 / 99536227 = B_2 \left( 0.03214 \cos \theta_2 - 0.15092 \sin \theta_2 \right)
\end{aligned}
\right\} \implies \left\{ \!\!\!\!\!\!\!\!\!\!\!\!
\begin{aligned}
\theta_2 & = -81.9756^\circ \\
\qquad B_2 & = -0.05169
\end{aligned}
\right.
$$

<!-- $$ B_2 = -\frac{102600}{14219461 \cos \theta_2} $$ -->
<!-- $$
\begin{aligned}
-\frac{791943}{99536227}
& = -\frac{102600}{14219461 \cos \theta_2} \left( 0.03214 \cos \theta_2 - 0.15092 \sin \theta_2 \right) \\
& = -\frac{102600}{14219461} \left( 0.03214 - 0.15092 \tan \theta_2 \right)
\end{aligned}
$$ -->
<!-- $$ \tan \theta_2 = \frac{0.03214 - \frac{263981}{239400}}{0.15092} $$ -->

Therefore, the inverse Laplace transform of the second term is:

$$ \mathcal{L}^{-1}\left\{ \frac{-\frac{86184000}{14219461} s - \frac{95033160}{14219461}}{840 s^2 + 54 s + 20} \right\} = -0.05169 \, \mathrm{e}^{-0.03214 t} \cos(0.15092 \, t - 81.9756^\circ) \, u(t) $$

In [ ]:
alpha_2 = 9 / 280
print(f"{alpha_2 = :.5f}")
omega_2 = sqrt(1 / 42 - (9 / 280)**2)
print(f"{omega_2 = :.5f} rad/s")
v_b_phase = theta_2 = float(atan((alpha_2 - A_4_sol / A_3_sol) / omega_2))
print(f"theta_2 = {np.rad2deg(theta_2):.4f} deg")
v_b_amplitude = B_2 = (A_3_sol / 840) / cos(theta_2)
print(f"{B_2 = :.5f} V")

And finally, substituting in the inverse Laplace transforms of the first and second terms yields the following full solution for $v_b(t)$ in the time domain:

$$ v_b(t) = (\underbrace{0.007635 \cos(3 t - 19.092^\circ)}_{\text{Steady-state response, }v_b^{SS}(t)} - \underbrace{0.05169 \, \mathrm{e}^{-0.03214 t} \cos(0.15092 \, t - 81.9756^\circ)}_\text{Transient response}) \, u(t) $$

From this we can calculate $v_a(t)$ via trigonometric identities. Its transient response is the same as for $v_b(t)$, and its steady-state response is:

$$
\begin{aligned}
v_a^{SS}(t)
& = v_{s2}(t) + v_b^{SS}(t) \\
& = 0.007635 \cos(3t - 19.092) + 2 \sin(3t) \\
% & = 0.007635 (\cos(3t) \cos(-19.092) + \sin(3t) \sin(-19.092)) + 2 \sin(3t) \\
% & = 0.007635 \cos(-19.092) \cos(3t) + (0.007635 \sin(-19.092) + 2) \sin(3t) \\
% & = \sqrt{(0.007635 \cos(-19.092))^2 + (0.007635 \sin(-19.092) + 2)^2} \\
% & \times \cos \left( 3 t + \arctan \left( -\frac{0.007635 \sin(-19.092) + 2}{0.007635 \cos(-19.092)} \right) \right) \\
& = 1.99752 \cos(3 t - 89.7930^\circ)
\end{aligned}
$$

In [ ]:
a = B_1 * cos(theta_1)
b = B_1 * sin(theta_1) + 2
v_a_amplitude = sqrt(a**2 + b**2)
v_a_phase = atan(-b / a)
print(f"{v_a_amplitude = :.5f} V")
print(f"v_a_phase = {np.rad2deg(float(v_a_phase)):.4f} deg")

In [ ]:
tr = B_2 * exp(-alpha_2 * t) * cos(omega_2 * t + theta_2)
v_a = v_a_amplitude * cos(3 * t + v_a_phase) + tr
v_b = v_b_amplitude * cos(3 * t + v_b_phase) + tr

ts = np.linspace(0, 100, 5000)

plt.plot(ts, lambdify(t, v_a)(ts), label=r"$v_a(t)$")
plt.plot(ts, lambdify(t, 10 * v_b)(ts), label=r"$10 v_b(t)$")
plt.legend(loc="lower right")
plt.savefig("img/fig_3_3.png", dpi=200)

```{image} img/fig_3_3.png
:align: center
:width: 65%
```

The solution for $v_b(t)$ (and $v_a(t)$) has two distinct terms. It is a linear combination of cosine waves, each with their own natural frequency and phase, but one of the cosine waves is exponentially decaying. The exponentially decaying cosine models how the circuit responds when turned on at $t = 0$---its *transient response*. The non-decaying cosine is how the circuit responds in the longer term---its *steady-state response* as $t$ tends to infinity (but practically much sooner as the transient quickly dies out).

## Solving the circuit with phasors

Using KCL, the sum of currents flowing out of supernode $ab$ must sum to zero. We can therefore write:

$$ - \mathbf{i}_\mathrm{L} + \mathbf{i}_\mathrm{R2} - \mathbf{i}_\mathrm{R1} + \mathbf{i}_\mathrm{C} = 0 $$

$$ - \frac{\mathbf{v}_\mathrm{L}}{Z_\mathrm{L}} + \frac{\mathbf{v}_\mathrm{R2}}{Z_\mathrm{R2}} - \frac{\mathbf{v}_\mathrm{R1}}{Z_\mathrm{R1}} + \frac{\mathbf{v}_\mathrm{C}}{Z_\mathrm{C}} = 0 $$

Substituting in the components' impedances and putting everything in terms of the node voltages gives us:

$$ - \frac{\mathbf{v}_{s1} - \mathbf{v}_a}{\mathbf{j} \, \omega L} + \frac{\mathbf{v}_a}{R_2} - \frac{\mathbf{v}_{s1} - \mathbf{v}_b}{R_1} + \frac{\mathbf{v}_b}{\mathbf{j} \frac{-1}{\omega C}} = 0 $$

<!-- $$ \mathbf{j} \frac{\mathbf{v}_{s1} - \mathbf{v}_a}{\omega L} + \frac{\mathbf{v}_a}{R_2} - \frac{\mathbf{v}_{s1} - \mathbf{v}_b}{R_1} + \mathbf{j} \, \omega C \mathbf{v}_b = 0 $$ -->

Paired with Equation {eq}`v_s2_eq`, $\mathbf{v}_a - \mathbf{v}_b = \mathbf{v}_{s2}$, this represents a system of two linear equations in two unknowns $\mathbf{v}_a$ and $\mathbf{v}_b$. Substituting Equation {eq}`v_s2_eq` into the above yields:

$$ \mathbf{j} \frac{\mathbf{v}_{s1} - (\mathbf{v}_{s2} + \mathbf{v}_b)}{\omega L} + \frac{\mathbf{v}_{s2} + \mathbf{v}_b}{R_2} - \frac{\mathbf{v}_{s1} - \mathbf{v}_b}{R_1} + \mathbf{j} \, \omega C \mathbf{v}_b = 0 $$

Solving for $\mathbf{v}_b$ yields the following, and we can solve for $\mathbf{v}_a$ from this using Equation {eq}`v_s2_eq`.

In [ ]:
v_b = symbols("v_b")

v_s1 = j * V_s1
v_s2 = j * V_s2

v_a = v_s2 + v_b
v_L = v_s1 - v_a
v_R2 = v_a
v_R1 = v_s1 - v_b
v_C = v_b

Z_L = j * w * L
Z_R2 = R_2
Z_R1 = R_1
Z_C = - j / (w * C)

i_L = v_L / Z_L
i_R2 = v_R2 / Z_R2
i_R1 = v_R1 / Z_R1
i_C = v_C / Z_C

eq_ph = Eq(- i_L + i_R2 - i_R1 + i_C, 0)
[v_b_sol] = solve(eq_ph, v_b)
v_b_sol

$$ \mathbf{v}_b = \frac{\omega L (R_2 \mathbf{v}_{s1} - R_1 \mathbf{v}_{s2}) + \mathbf{j} \, R_1 R_2 (\mathbf{v}_{s2} - \mathbf{v}_{s1})}{\omega L (R_1 + R_2) + \mathbf{j} \, (\omega^2 C L - 1) R_1 R_2} $$

Getting here required some algebra involving complex numbers. However, when solving a real circuit whose parameters have known values, this process would have been aided significantly by being able to evaluate/simplify expressions at each step. Let's introduce some very arbitrary values to see how $\mathbf{v}_b$ and $\mathbf{v}_a$ can be evaluated.

$$
\begin{aligned}
\mathbf{v}_b
& = \frac{(3 \ \mathrm{rad} / \mathrm{s}) (6 \ \mathrm{H}) ((5 \ \Omega) (\mathbf{j} \, 1 \ \mathrm{V}) - (4 \ \Omega) (\mathbf{j} \, 2 \ \mathrm{V})) + \mathbf{j} \, (4 \ \Omega) (5 \ \Omega) ((\mathbf{j} \, 2 \ \mathrm{V}) - (\mathbf{j} \, 1 \ \mathrm{V}))}{(3 \ \mathrm{rad} / \mathrm{s}) (6 \ \mathrm{H}) ((4 \ \Omega) + (5 \ \Omega)) + \mathbf{j} \, ((3 \ \mathrm{rad} / \mathrm{s})^2 (7 \ \mathrm{F}) (6 \ \mathrm{H}) - 1) (4 \ \Omega) (5 \ \Omega)} \\
& = \frac{-20 - \mathbf{j} \, 54}{162 + \mathbf{j} \, 7540} \ \mathrm{V} \\
& = \frac{-20 - \mathbf{j} \, 54}{162 + \mathbf{j} \, 7540} \, \frac{162 - \mathbf{j} \, 7540}{162 - \mathbf{j} \, 7540} \ \mathrm{V} \\
& = \frac{(-20 - \mathbf{j} \, 54)(162 - \mathbf{j} \, 7540)}{162^2 + 7540^2} \ \mathrm{V} \\
& = \frac{-3240 + \mathbf{j} \, 150800 - \mathbf{j} \, 8748  - 407160}{26244 + 56851600} \ \mathrm{V} \\
& = \frac{-410400 + \mathbf{j} \, 142052}{56877844} \ \mathrm{V} \\
& = \boxed{(-0.007215 + \mathbf{j} \, 0.002497) \ \mathrm{V}} \\\\
\mathbf{v}_a
& = \mathbf{v}_b + \mathbf{v}_{s2} \\
& = (-0.007215 + \mathbf{j} \, 0.002497) \ \mathrm{V} + \mathbf{j} \, 2 \ \mathrm{V} \\
& = \boxed{(-0.007215 + \mathbf{j} \, 2.002497) \ \mathrm{V}}
\end{aligned}
$$

In [ ]:
# Checking the result.

v_a_sol = v_a.subs(v_b, v_b_sol)

print(subs(v_b_sol).evalf(6))
print(subs(v_a_sol).evalf(6))

By calculating the amplitude $A$ and phase $\theta$, we can convert from complex number phasor notation $a + \mathbf{j} \, b$ to either polar phasor notation $A \angle \theta$ or back into the time domain as $A \cos(\omega t + \theta)$. The amplitude and phase are calculated using:

$$ A = \sqrt{a^2 + b^2}, \qquad \theta = \arctan \frac{b}{a} $$

In [ ]:
v_b_real, v_b_imag = subs(v_b_sol).as_real_imag()
v_a_real, v_a_imag = subs(v_a_sol).as_real_imag()

v_b_amplitude = float(sqrt(v_b_real**2 + v_b_imag**2))
v_a_amplitude = float(sqrt(v_a_real**2 + v_a_imag**2))
print(f"v_b_amplitude = {v_b_amplitude:.6f} V")
print(f"v_a_amplitude = {v_a_amplitude:.6f} V")

v_b_phase = float(atan(v_b_imag / v_b_real))
v_a_phase = float(atan(v_a_imag / v_a_real))
print(f"v_b_phase = {np.rad2deg(v_b_phase):.3f} deg")
print(f"v_a_phase = {np.rad2deg(v_a_phase):.3f} deg")

$$
\begin{aligned}
    \mathbf{v}_b & = 0.007635 \, \angle {-19.092}^\circ \\
    \mathbf{v}_a & = 2.002510 \, \angle {-89.794}^\circ
\end{aligned}
$$

In [161]:
# Converting from the phasor domain to the time domain.

v_b_sol = v_b_amplitude * cos(subs(w) * t + v_b_phase)
v_a_sol = v_b_sol + subs(v_s2 := V_s2 * sin(w * t))

With the amplitude and phase calculated, it becomes very easy to plot the solution for $v_a(t)$ and $v_b(t)$:

In [ ]:
ts = np.linspace(0, 10, 500)

plt.plot(ts, lambdify(t, v_a_sol)(ts), label="$v_a(t)$")
plt.plot(ts, lambdify(t, 100 * v_b_sol)(ts), label="$100 v_b(t)$")
plt.legend(loc="lower right")
plt.savefig("img/fig_3_4.png", dpi=200)

```{image} img/fig_3_4.png
:align: center
:width: 65%
```

## Checking the phasor solution

At this point, we have derived the equations representing the circuit with and without phasors, and we have solved the circuit with phasors. To check our work, we can plug the solution obtained using phasors into the differential equation obtained without using phasors, and verify that the equation holds true (that the left-hand side equals the right-hand side). This can be done conveniently by plotting both the left- and right-hand sides:

In [ ]:
# Visually checking that v_b solves the system of differential equations.

v_bp_sol = v_b_sol.diff(t)
v_bpp_sol = v_bp_sol.diff(t)

ts = np.linspace(0, 10, 500)

plt.plot(ts, lambdify(t, v_bpp_sol)(ts), label="Left-Hand Side = $v_b''(t)$")
rhs = subs(eq2.rhs).subs(
    [
        (Function("v_b")(t), v_b_sol),
        (Function("v_bp")(t), v_bp_sol),
    ]
)
plt.plot(ts, lambdify(t, rhs)(ts), label="Right-Hand Side", linestyle="dotted")
plt.legend(loc="lower right")
plt.savefig("img/fig_3_5.png", dpi=200)

```{image} img/fig_3_5.png
:align: center
:width: 65%
```

As can be seen, the left- and right-hand sides are equal.